Cluster images based on color features

In [1]:
from pyspark import SparkContext
from pyspark import SparkConf
# from pyspark.mllib.feature import HashingTF, IDF
# from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.clustering import KMeans, KMeansModel
# from StringIO import StringIO
# from PIL import Image
import numpy as np
# import csv
import os, tempfile
# import boto
import datetime

In [ ]:
# sc = SparkContext()

In [2]:
# AWS S3 credentials:

AWS_KEY = ""
AWS_SECRET = ""
sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", AWS_KEY)
sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", AWS_SECRET)

In [42]:
c_id = 3

In [43]:
directory = 's3n://amlyelp/subset/F%d_trainfinal.csv' % c_id

In [44]:
data = sc.textFile(directory)

In [1]:
# data.take(1)

In [46]:
data_parsed = data.map(lambda x: x.split(','))\
                  .map(lambda x: (x[0], np.array(x[1].split(' ')).astype(np.float)))\
                  .repartition(120).cache()
# del data

In [47]:
data_parsed.count()

61718

In [48]:
len(data_parsed.values().take(2)[1])

9000

In [49]:
clusters = KMeans.train(data_parsed.values(), 50, maxIterations=50)

In [50]:
# clusters.save(sc, '/home/hadoop/test_kmeans')
clusters.save(sc, ('s3n://amlyelp/subset/model/kmeans/color_feature_%d_' % c_id)\
              +str(datetime.datetime.now()).replace(' ', '_').replace(':','_')+'/')
# clusters = KMeansModel.load(sc, 'model/test_kmeans')

In [51]:
image_cluster = data_parsed.map(lambda x: (x[0],clusters.predict(x[1])))

In [52]:
image_cluster.count()

61718

In [53]:
image_cluster.saveAsTextFile("s3n://amlyelp/subset/image_cluster_result/image_cluster_color_feature_%d_%s" % \
                             (c_id, str(datetime.datetime.now()).replace(' ', '_').replace('.','_').replace(':','_')))